In [0]:
import torch

In [2]:
# do I have GPU:
gpu_available = torch.cuda.is_available()
print(gpu_available)

# number of GPUs I have:
num = torch.cuda.device_count()
print(f'I have {num} GPUs')

if num > 0:
    # current device index
    idx = torch.cuda.current_device()
    print(f'My current device has index {idx}')

    # GPU's name
    name = torch.cuda.get_device_name(idx)
    print(f'My GPU is {name}')

True
I have 1 GPUs
My current device has index 0
My GPU is Tesla P100-PCIE-16GB


In [3]:
%%bash
pip install transformers

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd

input_data = pd.read_csv("drive/My Drive/types_dataset_utf8.csv")
input_data = input_data.fillna(method="ffill")
input_data.tail(10)

words_list = list(set(input_data["Word"].values))
print(len(words_list))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


96556


In [7]:
# copied from Matt's notebook

class RetrieveSentance(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        function = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                     s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(function)
        self.sentences = [s for s in self.grouped]
    
    def retrieve(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

flatten = lambda l: [item for sublist in l for item in sublist]

sentences = RetrieveSentance(input_data)
# sentences_list = [" ".join([s[0] for s in sent]) for sent in sentences.sentences]

tag_vocab = list(set(flatten([[s[1] for s in sent] for sent in sentences.sentences])))
print(len(tag_vocab))

12265


In [8]:
from transformers import BertTokenizer, BertForTokenClassification, AdamW
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
import logging
import sys
from torch.utils.data import TensorDataset
from collections import defaultdict
from tqdm import tqdm

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger(__name__)

class InputExample(object):
    """A single training/test example for token classification."""

    def __init__(self, guid, words, labels):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            words: list. The words of the sequence.
            labels: (Optional) list. The labels for each word of the sequence. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.words = words
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids, position_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids
        self.position_ids = position_ids


def read_example(example):
    guid_index = 1
    examples = []
    words = []
    labels = []
    for line in example.split('\n'):
        splits = line.split(" ")
        words.append(splits[0])
        labels.append(splits[-1].replace("\n", ""))
    if words:
        examples.append(InputExample(guid="{}".format(guid_index),
                                        words=words,
                                        labels=labels))
    return examples


label_map = {label: i for i, label in enumerate(tag_vocab)}
o_idx = label_map['O']


def convert_examples_to_features(examples,
                                 label_list,
                                 max_seq_length,
                                 tokenizer,
                                 cls_token_at_end=False,
                                 cls_token="[CLS]",
                                 cls_token_segment_id=1,
                                 sep_token="[SEP]",
                                 sep_token_extra=False,
                                 pad_on_left=False,
                                 pad_token=0,
                                 pad_token_segment_id=0,
                                 pad_token_label_id=-1,
                                 sequence_a_segment_id=0,
                                 mask_padding_with_zero=True):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    # label_map = {label: i for i, label in enumerate(label_list)}
    # unk_idx = label_list.index('<unk>')
    # # if a label isn't found, default to <unk>
    # # we assume the label_list provided to the function already has <unk> included
    # label_map = defaultdict(lambda: unk_idx)
    # for i, label in enumerate(label_list):
    #     label_map[label] = i

    features = []
    for (ex_index, example) in tqdm(enumerate(examples)):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d of %d", ex_index, len(examples))

        tokens = []
        label_ids = []
        position_ids = []
        curr_position = 1
        for word, label in zip(example.words, example.labels):
            word_tokens = tokenizer.tokenize(word)
            tokens.extend(word_tokens)
            # Use the real label id for the first token of the word, and padding ids for the remaining tokens
            label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))
            position_ids.extend([curr_position] * len(word_tokens))
            curr_position += 1

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = 3 if sep_token_extra else 2
        if len(tokens) > max_seq_length - special_tokens_count:
            tokens = tokens[:(max_seq_length - special_tokens_count)]
            label_ids = label_ids[:(max_seq_length - special_tokens_count)]
            position_ids = position_ids[:(max_seq_length - special_tokens_count)]

        tokens += [sep_token]
        label_ids += [pad_token_label_id]
        position_ids += [0]
        if sep_token_extra:
            # roberta uses an extra separator b/w pairs of sentences
            tokens += [sep_token]
            label_ids += [pad_token_label_id]
            position_ids += [0]
        segment_ids = [sequence_a_segment_id] * len(tokens)

        if cls_token_at_end:
            tokens += [cls_token]
            label_ids += [pad_token_label_id]
            position_ids += [0]
            segment_ids += [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            label_ids = [pad_token_label_id] + label_ids
            position_ids = [0] + position_ids
            segment_ids = [cls_token_segment_id] + segment_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
            segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            label_ids = ([pad_token_label_id] * padding_length) + label_ids
            position_ids = ([0] * padding_length) + position_ids
        else:
            input_ids += ([pad_token] * padding_length)
            input_mask += ([0 if mask_padding_with_zero else 1] * padding_length)
            segment_ids += ([pad_token_segment_id] * padding_length)
            label_ids += ([pad_token_label_id] * padding_length)
            position_ids += ([0] * padding_length)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(label_ids) == max_seq_length
        assert len(position_ids) == max_seq_length

        if ex_index < 3:
            logger.info("*** Example ***")
            logger.info("guid: %s", example.guid)
            logger.info("tokens: %s", " ".join([str(x) for x in tokens]))
            logger.info("input_ids: %s", " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s", " ".join([str(x) for x in input_mask]))
            logger.info("segment_ids: %s", " ".join([str(x) for x in segment_ids]))
            logger.info("label_ids: %s", " ".join([str(x) for x in label_ids]))
            logger.info("position_ids: %s", " ".join([str(x) for x in position_ids]))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_ids=label_ids,
                              position_ids=position_ids))
    return features


def convert_features_to_dataset(features):
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)
    all_position_ids = torch.tensor([f.position_ids for f in features], dtype=torch.long)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids, all_position_ids)
    return dataset

In [10]:
import random

def create_dataset(sentences):
    examples = []
    idx = 1
    for example in sentences:
        tokens, tags = preprocess_line(example)
        examples.append(InputExample(str(idx), tokens, tags))
        idx += 1

    features = convert_examples_to_features(examples, list(tag_vocab), 512, tokenizer, pad_token_label_id=o_idx)
    for feature in features:
        assert(all([idx >= 0 and idx < len(tag_vocab) for idx in feature.label_ids]))

    dataset = convert_features_to_dataset(features)
    return dataset

def preprocess_line(sentence):
    tokens = [s[0] for s in sentence]
    tags = [s[1] for s in sentence]
    return (tokens, tags)

num_examples = len(sentences.sentences)
random.shuffle(sentences.sentences)
test_count = int(num_examples * 0.1)
half_count = int(num_examples * 0.5) # let's try just using half of the dataset?
test_data_examples = sentences.sentences[:test_count]
train_data_examples = sentences.sentences[test_count:half_count]

train_dataset = create_dataset(train_data_examples)
print('done parsing training set')
test_dataset = create_dataset(test_data_examples)
print('done parsing test set')

0it [00:00, ?it/s]

INFO:__main__:Writing example 0 of 5337
INFO:__main__:*** Example ***
INFO:__main__:guid: 1
INFO:__main__:tokens: [CLS] import { it ##ele ##metry ##ite ##m } from " s " ; import { in ##ot ##ification ##list ##ener } from " s " ; export interface in ##ot ##ification ##mana ##ger { listeners : in ##ot ##ification ##list ##ener [ ] ; add ##not ##ification ##list ##ener ( listener ) : void ; remove ##not ##ification ##list ##ener ( listener ) : void ; events ##sen ##t ( events : it ##ele ##metry ##ite ##m [ ] ) : void ; events ##dis ##card ##ed ( events : it ##ele ##metry ##ite ##m [ ] , reason ) : void ; } [SEP]
INFO:__main__:input_ids: 101 12324 1063 2009 12260 24327 4221 2213 1065 2013 1000 1055 1000 1025 12324 1063 1999 4140 9031 9863 24454 1065 2013 1000 1055 1000 1025 9167 8278 1999 4140 9031 24805 4590 1063 13810 1024 1999 4140 9031 9863 24454 1031 1033 1025 5587 17048 9031 9863 24454 1006 19373 1007 1024 11675 1025 6366 17048 9031 9863 24454 1006 19373 1007 1024 11675 1025 2824 505

5337it [04:25, 20.08it/s]
0it [00:00, ?it/s]

done parsing training set
INFO:__main__:Writing example 0 of 1334
INFO:__main__:*** Example ***
INFO:__main__:guid: 1
INFO:__main__:tokens: [CLS] export interface add ##back ##ground ##im ##ga ##ction { type : ' s ' , background ##im ##g : { ur ##l : string key : string } } export interface set ##back ##ground ##im ##ga ##ction { type : ' s ' , background ##im ##g : { ur ##l : string key : string } [ ] } export con ##st add ##back ##ground ##im ##g = ( background ##im ##g , key ) = > { return { type : ' s ' , background ##im ##g : { ur ##l : background ##im ##g , key } } } export con ##st set ##back ##ground ##im ##g = ( background ##im ##g : { ur ##l : string key : string } [ ] ) = > { return { type : ' s ' , background ##im ##g : background ##im ##g } } [SEP]
INFO:__main__:input_ids: 101 9167 8278 5587 5963 16365 5714 3654 7542 1063 2828 1024 1005 1055 1005 1010 4281 5714 2290 1024 1063 24471 2140 1024 5164 3145 1024 5164 1065 1065 9167 8278 2275 5963 16365 5714 3654 7542 1063 2828 1

1334it [01:06, 20.12it/s]


done parsing test set


In [0]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def calculate_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    nonzero = labels_flat != o_idx
    # print(pred_flat, len(pred_flat))
    # print(labels_flat, len(labels_flat))
    # print('pred_flat', pred_flat)
    # print('labels_flat', labels_flat)
    # print('nonzero_pred_flat', pred_flat[nonzero])
    # print('nonzero_labels_flat', labels_flat[nonzero])
    return np.sum(pred_flat[nonzero] == labels_flat[nonzero]) / len(labels_flat[nonzero])

In [12]:
%%capture
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(tag_vocab))
model.to(device)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/bert-base-uncased-config.json HTTP/1.1" 200 0
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:transformers.configuration_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 12265,
  "output_attentions": false,
  "output_hidden_states": false

In [13]:
from torch.utils.data import DataLoader

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": 0.01},
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=0.00003)

batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
epochs = 3

for _ in range(epochs):
    model.train() # set model to training mode
    train_loss = 0
    train_examples, train_steps = 0, 0
    for step, batch in tqdm(enumerate(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0],
                  "attention_mask": batch[1],
                  "labels": batch[3],
                  "position_ids": batch[4]}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs[0]
        train_loss += loss.item()
        train_examples += batch[0].size(0)
        train_steps += 1
        loss.backward()
        optimizer.step()
    print('Train loss:', train_loss/train_steps)

    model.eval() # set model to eval mode
    eval_acc, eval_steps = 0, 0

    for step, batch in enumerate(test_dataloader):
        batch = tuple(t.to(device) for t in batch)
        # inputs = {"input_ids": batch[0],
        #           "attention_mask": batch[1],
        #           "labels": batch[3],
        #           "position_ids": batch[4]}
        with torch.no_grad():
            output = model(input_ids=batch[0], attention_mask=batch[1], position_ids=batch[4])
        logits = output[0]
        logits = logits.detach().cpu().numpy()
        label_ids = batch[3].to('cpu').numpy()
        eval_acc += calculate_accuracy(logits, label_ids)
        eval_steps += 1

    print('Eval accuracy:', eval_acc/eval_steps)

1335it [06:13,  3.57it/s]


Train loss: 0.67773325543167
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero

0it [00:00, ?it/s]

[5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
Eval accuracy: 0.9690394601422155


1335it [06:14,  3.57it/s]


Train loss: 0.327328249744457
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzer

0it [00:00, ?it/s]

pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
Eval accuracy: 0.9768022548652695


1335it [06:13,  3.58it/s]


Train loss: 0.2583092235529286
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_labels_flat [5123 5123 5123 ... 5123 5123 5123]
pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
nonzero_pred_flat [5123 5123 5123 ... 5123 5123 5123]
nonze

In [20]:
# run evaluation separately if we want (without retraining model)
# by running this cell

model.eval() # set model to training mode
for step, batch in enumerate(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # inputs = {"input_ids": batch[0],
    #           "attention_mask": batch[1],
    #           "labels": batch[3],
    #           "position_ids": batch[4]}
    with torch.no_grad():
        output = model(input_ids=batch[0], attention_mask=batch[1], position_ids=batch[4])
    logits = output[0]
    logits = logits.detach().cpu().numpy()
    label_ids = batch[3].to('cpu').numpy()
    eval_acc += calculate_accuracy(logits, label_ids)
    eval_steps += 1

    # preview what a 
    if step == 0:
        token_ids = batch[0]
        pred_flat = np.argmax(logits, axis=2).flatten()
        print('pred_flat', pred_flat)
        labels_flat = label_ids.flatten()
        print('labels_flat', labels_flat)
        nonzero = labels_flat != o_idx
        print('pred tags', [tag_vocab[idx] for idx in pred_flat])
        print('label tags', [tag_vocab[idx] for idx in labels_flat])
        print('cleaned:')
        print('pred tags', [tag_vocab[idx] for idx in pred_flat[nonzero]])
        print('label tags', [tag_vocab[idx] for idx in labels_flat[nonzero]])
print('Eval accuracy:', eval_acc/eval_steps)

pred_flat [5123 5123 5123 ... 5123 5123 5123]
labels_flat [5123 5123 5123 ... 5123 5123 5123]
pred tags ['O', 'O', 'O', '$any$', 'O', 'O', 'O', 'O', 'O', 'O', '$string$', 'O', 'O', 'O', 'O', 'O', '$complex$', 'O', 'O', 'O', 'O', '$string$', 'O', 'O', 'O', '$string$', 'O', 'O', 'O', 'O', 'O', 'O', '$any$', 'O', 'O', 'O', 'O', 'O', 'O', '$string$', 'O', 'O', 'O', 'O', 'O', '$complex$', 'O', 'O', 'O', 'O', '$string$', 'O', 'O', 'O', '$string$', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '$any$', 'O', 'O', 'O', 'O', 'O', 'O', '$string$', 'O', 'O', 'O', '$string$', 'O', 'O', 'O', 'O', 'O', 'O', '$string$', 'O', 'O', 'O', 'O', 'O', '$complex$', 'O', 'O', 'O', 'O', '$string$', 'O', 'O', 'O', 'O', 'O', 'O', '$string$', 'O', 'O', 'O', 'O', 'O', 'O', '$void$', 'O', 'O', 'O', 'O', 'O', 'O', '$complex$', 'O', 'O', 'O', 'O', '$string$', 'O', 'O', 'O', '$string$', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '$string$', 'O', 'O', 'O', 'O', 'O', '$string$', 'O', 'O', 'O', 'O', 'O', 'O', '